<a href="https://colab.research.google.com/github/khanhtimn/Project-III/blob/main/refrag_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello")

hello
